In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from m3util.viz.arrows import DrawArrow
from m3util.viz.layout import layout_subfigures_inches
from m3util.viz.printing import printer
import matplotlib.pyplot as plt
from m3util.viz.style import set_style
from m3util.viz.images import display_image

import matplotlib.image as mpimg
from belearn.dataset.dataset import BE_Dataset
from m3util.util.IO import download_and_unzip
from belearn.viz.viz import Viz

set_style("printing")

printing = printer(basepath="./Figures/")

In [ ]:
# Download the data file from Zenodo
url = 'https://zenodo.org/record/7774788/files/PZT_2080_raw_data.h5?download=1'

# Specify the filename and the path to save the file
filename = '/data_raw.h5'
save_path = './Data'

# download the file
download_and_unzip(filename, url, save_path)

In [ ]:
data_path = save_path + '/' + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

In [ ]:
import numpy as np

# insatiate the visualization object
image_scalebar = [2000, 500, "nm", "br"]

BE_viz = Viz(
    dataset,
    printing,
    verbose=True,
    SHO_ranges=[(0, 1.5e-4), (1.31e6, 1.33e6), (-300, 300), (-np.pi, np.pi)],
    image_scalebar=image_scalebar,
)


In [ ]:
prediction = {"resampled": False, "label": "Raw"}


BE_viz.raw_data_comparison(prediction, pixel=3000, voltage_step=60, filename="Figure_2_1_raw_cantilever_response")


In [ ]:
from m3util.viz.arrows import DrawArrow
from m3util.viz.layout import layout_subfigures_inches
import matplotlib.pyplot as plt
from m3util.viz.style import set_style


pixel = 1500

# Dictionary defining the subfigures with positions in inches
subfigures_dict = {
    "AFM_Image": {
        "position": (0, 6.5 - 3.2131, 2.0972 - 18*2 / 72, 2.0972- 18*2 / 72),
        "skip_margin": True,
    },
    "real_imag": {"position": (2.2014 -(18*2)/72, 6.5 - 2.1364, 2.0972, 2.0972)},
    "amp_phase": {"position": (4.3588, 6.5 - 2.1364, 2.0972, 2.0972)},
    "BE": {"position": (2.2014, 6.5 - 4.2942, 2.0972, 2.0972)},
    "FFT": {"position": (4.3588, 6.5 - 4.2942, 2.0972, 2.0972)},
    "Bipolar": {"position": (0.0532, 6.5 - 6.4609, 4.2361, 2.0972)},
    "Hysteresis": {"position": (4.3588, 6.5 - 6.4609, 2.0972, 2.0972)},
}



# Create a custom figure of size (10 inches width, 6 inches height)
fig, ax = layout_subfigures_inches(size=(6.5, 6.5), subfigures_dict=subfigures_dict)

arrow = DrawArrow(
    fig,
    start_pos=(1.0926 - 2 * 18 / 72, 6.5 - 1 - 2 * 18 / 72),
    end_pos=(2.2014 - 2 * 18 / 72, 6.5 - 0.4972 - 2 * 18 / 72),
    text="FFT",
    text_position="center",
    text_alignment="center",
    vertical_text_displacement="top",
)

display_image(ax["AFM_Image"], "assets/AFM_field.png")

arrow.draw()

true = {"resampled": False, "label": "Raw"}

BE_viz.plot_real_imainary(ax["real_imag"], true, pixel=3000, voltage_step=60)



In [ ]:
arrow._get_text_position()